# 🛰️ EuroSAT: Uydu Görüntüleri ile Arazi Sınıflandırma

Bu notebook, Sentinel-2 uydu görüntülerini kullanarak arazi sınıflandırması yapmak için hazırlanmıştır.

**İçerik:**
1. Kütüphanelerin Yüklenmesi
2. Veri Setinin Hazırlanması
3. Model A: Simple CNN (Baseline)
4. Model B: ResNet-18 (Feature Extraction)
5. Model C: ResNet-18 (Fine-Tuning)
6. Model Karşılaştırması ve Sonuçlar

## 1. Kütüphanelerin Yüklenmesi

In [ ]:
import sys
import os

# Proje kök dizinini ayarla (sabit yol)
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, PROJECT_ROOT)
sys.path.insert(0, os.path.join(PROJECT_ROOT, 'src'))

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Proje modülleri
from data_loader import download_eurosat, create_dataloaders, CLASS_NAMES, NUM_CLASSES, denormalize
from model import create_model
from train_utils import (train_model, evaluate_model, plot_training_history, 
                         plot_confusion_matrix, plot_class_performance, compare_models, load_checkpoint)

# Klasörleri oluştur
os.makedirs(os.path.join(PROJECT_ROOT, 'data'), exist_ok=True)
os.makedirs(os.path.join(PROJECT_ROOT, 'models'), exist_ok=True)
os.makedirs(os.path.join(PROJECT_ROOT, 'reports', 'figures'), exist_ok=True)

print(f"Proje Dizini: {PROJECT_ROOT}")
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Cihaz ayarı
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Kullanılan cihaz: {device}")

# Tekrarlanabilirlik için seed
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

## 2. Veri Setinin Hazırlanması

In [ ]:
# Veri setini indir
DATA_DIR = os.path.join(PROJECT_ROOT, 'data')
data_path = download_eurosat(data_dir=DATA_DIR, extract_dir=DATA_DIR)

In [ ]:
# Hiperparametreler
BATCH_SIZE = 64
IMG_SIZE = 224
NUM_WORKERS = 2 # Windows için 0

# DataLoader'ları oluştur
train_loader, val_loader, test_loader, dataset_info = create_dataloaders(
    data_path=data_path,
    batch_size=BATCH_SIZE,
    train_ratio=0.8,
    val_ratio=0.1,
    num_workers=NUM_WORKERS,
    img_size=IMG_SIZE,
    seed=SEED
)

In [ ]:
# Örnek görüntüleri görselleştir
images, labels = next(iter(train_loader))
images_denorm = denormalize(images)

fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.flatten()

for i in range(10):
    img = images_denorm[i].permute(1, 2, 0).numpy()
    img = np.clip(img, 0, 1)
    axes[i].imshow(img)
    axes[i].set_title(CLASS_NAMES[labels[i]], fontsize=10)
    axes[i].axis('off')

plt.suptitle('EuroSAT Veri Seti - Örnek Görüntüler', fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(PROJECT_ROOT, 'reports', 'figures', 'sample_images.png'), dpi=300)
plt.show()

## 3. Model A: Simple CNN (Baseline)

Sıfırdan tasarlanan 4 katmanlı CNN modeli.

In [ ]:
# Model A: Simple CNN
model_a = create_model('simple_cnn', num_classes=NUM_CLASSES)
model_a = model_a.to(device)

# Loss ve Optimizer
criterion = nn.CrossEntropyLoss()
optimizer_a = optim.Adam(model_a.parameters(), lr=0.001)
scheduler_a = optim.lr_scheduler.ReduceLROnPlateau(optimizer_a, mode='min', patience=2, factor=0.5)

In [ ]:
# Model A Eğitimi
history_a = train_model(
    model=model_a,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer_a,
    scheduler=scheduler_a,
    num_epochs=15,
    device=device,
    save_path=os.path.join(PROJECT_ROOT, 'models', 'simple_cnn_best.pth'),
    early_stopping_patience=5
)

In [ ]:
# Model A Eğitim Grafikleri
plot_training_history(history_a, save_path=os.path.join(PROJECT_ROOT, 'reports', 'figures', 'simple_cnn_training.png'))

In [ ]:
# Model A Değerlendirme
checkpoint_a = load_checkpoint(model_a, os.path.join(PROJECT_ROOT, 'models', 'simple_cnn_best.pth'))
metrics_a = evaluate_model(model_a, test_loader, device, CLASS_NAMES)

In [ ]:
# Model A Confusion Matrix
plot_confusion_matrix(
    metrics_a['true_labels'], 
    metrics_a['predictions'], 
    CLASS_NAMES,
    save_path=os.path.join(PROJECT_ROOT, 'reports', 'figures', 'simple_cnn_confusion_matrix.png')
)

## 4. Model B: ResNet-18 (Feature Extraction)

ImageNet üzerinde önceden eğitilmiş ResNet-18. Backbone dondurulmuş, sadece classifier eğitilir.

In [ ]:
# Model B: ResNet-18 (Frozen)
model_b = create_model('resnet18_frozen', num_classes=NUM_CLASSES, pretrained=True)
model_b = model_b.to(device)

optimizer_b = optim.Adam(filter(lambda p: p.requires_grad, model_b.parameters()), lr=0.001)
scheduler_b = optim.lr_scheduler.ReduceLROnPlateau(optimizer_b, mode='min', patience=2, factor=0.5)

In [ ]:
# Model B Eğitimi
history_b = train_model(
    model=model_b,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer_b,
    scheduler=scheduler_b,
    num_epochs=10,
    device=device,
    save_path=os.path.join(PROJECT_ROOT, 'models', 'resnet18_frozen_best.pth'),
    early_stopping_patience=5
)

In [ ]:
# Model B Eğitim Grafikleri
plot_training_history(history_b, save_path=os.path.join(PROJECT_ROOT, 'reports', 'figures', 'resnet18_frozen_training.png'))

In [ ]:
# Model B Değerlendirme
checkpoint_b = load_checkpoint(model_b, os.path.join(PROJECT_ROOT, 'models', 'resnet18_frozen_best.pth'))
metrics_b = evaluate_model(model_b, test_loader, device, CLASS_NAMES)

In [ ]:
# Model B Confusion Matrix
plot_confusion_matrix(
    metrics_b['true_labels'], 
    metrics_b['predictions'], 
    CLASS_NAMES,
    save_path=os.path.join(PROJECT_ROOT, 'reports', 'figures', 'resnet18_frozen_confusion_matrix.png')
)

## 5. Model C: ResNet-18 (Fine-Tuning)

Tüm katmanlar açılarak fine-tuning yapılır. En yüksek performans beklenir.

In [ ]:
# Model C: ResNet-18 (Fine-Tuning)
model_c = create_model('resnet18_finetune', num_classes=NUM_CLASSES, pretrained=True)
model_c = model_c.to(device)

optimizer_c = optim.Adam(model_c.parameters(), lr=0.0001)
scheduler_c = optim.lr_scheduler.ReduceLROnPlateau(optimizer_c, mode='min', patience=2, factor=0.5)

In [ ]:
# Model C Eğitimi
history_c = train_model(
    model=model_c,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer_c,
    scheduler=scheduler_c,
    num_epochs=15,
    device=device,
    save_path=os.path.join(PROJECT_ROOT, 'models', 'resnet18_finetune_best.pth'),
    early_stopping_patience=5
)

In [ ]:
# Model C Eğitim Grafikleri
plot_training_history(history_c, save_path=os.path.join(PROJECT_ROOT, 'reports', 'figures', 'resnet18_finetune_training.png'))

In [ ]:
# Model C Değerlendirme
checkpoint_c = load_checkpoint(model_c, os.path.join(PROJECT_ROOT, 'models', 'resnet18_finetune_best.pth'))
metrics_c = evaluate_model(model_c, test_loader, device, CLASS_NAMES)

In [ ]:
# Model C Confusion Matrix
plot_confusion_matrix(
    metrics_c['true_labels'], 
    metrics_c['predictions'], 
    CLASS_NAMES,
    save_path=os.path.join(PROJECT_ROOT, 'reports', 'figures', 'resnet18_finetune_confusion_matrix.png')
)

In [ ]:
# Model C Sınıf Bazlı Performans
plot_class_performance(
    metrics_c, 
    CLASS_NAMES,
    save_path=os.path.join(PROJECT_ROOT, 'reports', 'figures', 'resnet18_finetune_class_performance.png')
)

## 6. Model Karşılaştırması ve Sonuçlar

In [ ]:
# Tüm modelleri karşılaştır
all_results = {
    'Simple CNN': metrics_a,
    'ResNet-18 (Frozen)': metrics_b,
    'ResNet-18 (Fine-tuned)': metrics_c
}

compare_models(all_results, save_path=os.path.join(PROJECT_ROOT, 'reports', 'figures', 'model_comparison.png'))

In [ ]:
# Detaylı karşılaştırma tablosu
print("\n" + "="*100)
print("📊 DETAYLI MODEL KARŞILAŞTIRMA TABLOSU")
print("="*100)
print(f"{'Model':<25} {'Test Acc':<12} {'F1 (Macro)':<12} {'F1 (Weighted)':<14} {'Precision':<12} {'Recall':<12}")
print("-"*100)

for name, metrics in all_results.items():
    print(f"{name:<25} "
          f"{metrics['test_accuracy']*100:>10.2f}% "
          f"{metrics['f1_macro']:>11.4f} "
          f"{metrics['f1_weighted']:>13.4f} "
          f"{metrics['precision_macro']:>11.4f} "
          f"{metrics['recall_macro']:>11.4f}")

print("="*100)

In [ ]:
# Eğitim süreçlerini karşılaştır
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss karşılaştırması
axes[0].plot(history_a['val_loss'], label='Simple CNN', linewidth=2)
axes[0].plot(history_b['val_loss'], label='ResNet-18 (Frozen)', linewidth=2)
axes[0].plot(history_c['val_loss'], label='ResNet-18 (Fine-tuned)', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Validation Loss')
axes[0].set_title('Validation Loss Karşılaştırması')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy karşılaştırması
axes[1].plot([a*100 for a in history_a['val_acc']], label='Simple CNN', linewidth=2)
axes[1].plot([a*100 for a in history_b['val_acc']], label='ResNet-18 (Frozen)', linewidth=2)
axes[1].plot([a*100 for a in history_c['val_acc']], label='ResNet-18 (Fine-tuned)', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Validation Accuracy (%)')
axes[1].set_title('Validation Accuracy Karşılaştırması')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(PROJECT_ROOT, 'reports', 'figures', 'training_comparison.png'), dpi=300)
plt.show()

## 7. Sonuç

In [ ]:
# Final özet
print("\n" + "="*60)
print("🎯 PROJE SONUÇ ÖZETİ")
print("="*60)
print(f"\n✅ En İyi Model: ResNet-18 (Fine-Tuned)")
print(f"   - Test Accuracy: {metrics_c['test_accuracy']*100:.2f}%")
print(f"   - F1-Score (Macro): {metrics_c['f1_macro']:.4f}")
print(f"   - F1-Score (Weighted): {metrics_c['f1_weighted']:.4f}")
print(f"\n📁 Kaydedilen Dosyalar:")
print(f"   - {os.path.join(PROJECT_ROOT, 'models')}")
print(f"   - {os.path.join(PROJECT_ROOT, 'reports', 'figures')}")
print("="*60)



## 8. Gradio Arayüzü

In [ ]:
# 1. Önce Gradio'yu kuralım (Eğer kurulu değilse)
try:
    import gradio as gr
except ImportError:
    !pip install gradio
    import gradio as gr

import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import os
import pandas as pd
from model import create_model # Senin model.py dosyanı kullanıyoruz

# --- AYARLAR ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_names = [
    'AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial',
    'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake'
]

predict_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# --- MODEL YÜKLEME (Önceki mantığın aynısı) ---
def load_models_for_gradio():
    models_dict = {
        'Simple CNN': 'models/simple_cnn_best.pth',
        'ResNet (Frozen)': 'models/resnet18_frozen_best.pth',
        'ResNet (Fine-Tuned)': 'models/resnet18_finetune_best.pth'
    }

    loaded = {}
    print("Modeller yükleniyor, lütfen bekleyin...")

    for name, path in models_dict.items():
        full_path = os.path.join(PROJECT_ROOT, path) if 'PROJECT_ROOT' in globals() else path

        if not os.path.exists(full_path):
            print(f"⚠️ Uyarı: {name} bulunamadı ({full_path})")
            continue

        try:
            # Mimari seçimi
            if 'Simple' in name:
                model = create_model('simple_cnn', num_classes=10)
            elif 'Frozen' in name:
                model = create_model('resnet18_frozen', num_classes=10)
            else:
                model = create_model('resnet18_finetune', num_classes=10)

            model = model.to(device)

            # Ağırlıkları yükle
            checkpoint = torch.load(full_path, map_location=device)
            if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
                model.load_state_dict(checkpoint['model_state_dict'])
            else:
                model.load_state_dict(checkpoint)

            model.eval()
            loaded[name] = model
            print(f"✅ {name} yüklendi.")

        except Exception as e:
            print(f"❌ Hata ({name}): {e}")

    return loaded

# Modelleri hafızaya alalım
loaded_models = load_models_for_gradio()

# --- GRADIO TAHMİN FONKSİYONU ---
def predict_image(image):
    if image is None:
        return "Lütfen bir resim yükleyin.", pd.DataFrame()

    if not loaded_models:
        return "Modeller yüklenemedi! Dosya yollarını kontrol edin.", pd.DataFrame()

    # Resmi hazırla
    img_tensor = predict_transform(image).unsqueeze(0).to(device)

    results = []
    best_score = -1
    best_model_name = ""
    best_prediction = ""

    # Tüm modellerden tahmin al
    for model_name, model in loaded_models.items():
        with torch.no_grad():
            outputs = model(img_tensor)
            probs = torch.nn.functional.softmax(outputs[0], dim=0)
            confidence, idx = torch.max(probs, 0)

        pred_class = class_names[idx.item()]
        conf_val = confidence.item()

        # En iyiyi takip et
        if conf_val > best_score:
            best_score = conf_val
            best_model_name = model_name
            best_prediction = pred_class

        results.append([
            model_name,
            pred_class,
            f"%{conf_val * 100:.2f}"
        ])

    # Sonuç metni
    summary_text = (f"🏆 KAZANAN: {best_model_name}\n"
                    f"👉 Tahmin: {best_prediction}\n"
                    f"💪 Güven: %{best_score * 100:.2f}")

    # Pandas DataFrame oluştur
    df = pd.DataFrame(results, columns=["Model", "Tahmin", "Güven Skoru"])

    return summary_text, df

# --- ARAYÜZÜ OLUŞTUR ---
interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil", label="Uydu Görüntüsü Yükle (Sentinel-2 / Google Maps)"),
    outputs=[
        gr.Textbox(label="Sonuç Özeti"),
        gr.Dataframe(label="Detaylı Karşılaştırma Tablosu", headers=["Model", "Tahmin", "Güven Skoru"])
    ],
    title="🛰️ EuroSAT Arazi Sınıflandırma AI",
    description="Simple CNN, ResNet (Frozen) ve ResNet (Fine-Tuned) modellerini aynı görüntü üzerinde karşılaştırın.",
    theme="default"
)

# Arayüzü başlat (share=True derseniz 72 saatlik herkese açık link verir)
interface.launch(share=True, debug=True)